!pip install wikipedia
import wikipedia as wp
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

##!conda install -c conda-forge folium=0.5.0 --yes
##import folium # map rendering library

print("Libraries Imported!")

from lxml import html

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
request = requests.get(url)
file = html.fromstring(request.content)
tr_elements = file.xpath('//tr')
tr_elements

col = []
i = 0
for tr in tr_elements[0]:
    i += 1
    header = tr.text_content()
    col.append((header,[]))    
for index in range(1, len(tr_elements) - 4):
    tr = tr_elements[index]   
    i = 0  
    for t in tr.iterchildren():
        data = t.text_content()
        if data == "\n":
            i+=1
            pass
        else: 
            data = data[0:len(data)-1]
        col[i][1].append(data)
        i += 1
        
Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict) 

#df=df[df[1]!='Not assigned']

df.head()


In [145]:
import requests 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [152]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
request = requests.get(url).text

bs = BeautifulSoup(request, "xml")
table = bs.find("table")
columns = ["Postalcode","Borough","Neighborhood"]
df = pd.DataFrame(columns = columns)
for tr_cell in table.find_all("tr"):
    row_data = []
    for td_cell in tr_cell.find_all("td"):
        row_data.append(td_cell.text.strip())
    if len(row_data) == 3:
        df.loc[len(df)] = row_data
        
#drop if Borough is not assigned
df = df[df["Borough"] != "Not assigned"] 
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [161]:
#merge neighborhoods in the same postal code area
df_joined = df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df_joined = df_joined.reset_index(drop = False)
df_joined.rename(columns = {'Neighborhood':'Neighborhood_joined'}, inplace = True)

df1 = pd.merge(df, df_joined, on = 'Postalcode')
df1.drop(['Neighborhood'],axis = 1,inplace = True)
df1.drop_duplicates(inplace = True)
df1.rename(columns = {'Neighborhood_joined':'Neighborhood'},inplace=True)
df1.head(10)


,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [160]:
df1.shape

(103, 3)